# Inferencia de las fuerzas moleculares de los elementos del dataset QM9

## Importación de módulos

In [1]:
import torch
import numpy as np
import pandas as pd
import os
from scripts.qm9 import QM9

/root/mambaforge/envs/torchmd-net/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Definición de directorios

In [2]:
path_splits_npy = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/splits.npz"

# Ruta al dataset que uso torchmd-net
path_torchmdnet_dataset = "/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/dataset_qm9.pt"

# Ruta al modelo entrenado
path_trained_model="/workspace/tesis-potenciales-moleculares/inferencia-fuerzas/epoch=669-val_loss=0.0003-test_loss=0.0064.ckpt"

## Cargar modelo

In [3]:
from torchmdnet.models.model import load_model
model= load_model(path_trained_model, derivative=True)

## Cargar database

In [31]:
# Cargar archivo del dataset que utiliza torchmd-net
torchmdnet_dataset = torch.load(path_torchmdnet_dataset)

## Inferencias individuales

In [32]:
#número de indice de molecula
num_mol=10

In [42]:
molecula=torchmdnet_dataset[num_mol]

z_sample = molecula.z
pos_sample = molecula.pos
n_energy_sample = molecula.y
# Hacer la inferencia
n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
# Transformar la inferencia
#n_energy_inferred = float(n_energy_inferred)
#n_energy_sample = float(n_energy_sample)       

In [43]:
print(len(z_sample))
z_sample=z_sample.tolist()
z_sample=" ".join(map(str, z_sample))
print(z_sample)

7
6 6 8 1 1 1 1


In [38]:
pos_sample=pos_sample.flatten().tolist()
pos_sample=" ".join(map(str, pos_sample))
print(pos_sample)

-0.002899999963119626 1.5098999738693237 0.008700000122189522 0.026100000366568565 0.0032999999821186066 -0.03750000149011612 0.942300021648407 -0.6550999879837036 -0.45680001378059387 0.9228000044822693 1.926300048828125 -0.39149999618530273 -0.8619999885559082 1.878499984741211 -0.5648000240325928 -0.15049999952316284 1.8438999652862549 1.0428999662399292 -0.8944000005722046 -0.4864000082015991 0.357699990272522


In [39]:
n_energy_sample=float(n_energy_sample)
print(n_energy_sample)

-4184.77392578125


In [40]:
n_energy_inferred=float(n_energy_inferred)
print(n_energy_inferred) 

-4184.77587890625


In [41]:
n_forces_inferred=n_forces_inferred.flatten().tolist()
n_forces_inferred=" ".join(map(str, n_forces_inferred))
print(n_forces_inferred)

-0.20835821330547333 -0.7185043096542358 0.0755651667714119 0.09281303733587265 1.2005563974380493 -0.013574501499533653 0.34101352095603943 -0.08474434167146683 -0.15249693393707275 -0.06059858947992325 -0.1645474135875702 0.023261964321136475 0.1451825350522995 -0.23716601729393005 0.09191802144050598 0.025890318676829338 -0.2322511225938797 -0.17816679179668427 -0.3359425961971283 0.23665688931941986 0.15349306166172028


## Creación de un nuevo database con fuerzas inferidas

## Importación de módulos

In [ ]:
import numpy as np
import h5py
import random
import os
import shutil

## Creación de archivos temporales

In [ ]:
nombre_carpeta = "temp"

if os.path.exists(nombre_carpeta) and os.path.isdir(nombre_carpeta):
    shutil.rmtree(nombre_carpeta)
    os.mkdir(nombre_carpeta)
else:
    os.mkdir(nombre_carpeta)


os.mkdir("datos-procesados")

molecula=torchmdnet_dataset[num_mol]

z_sample = molecula.z
pos_sample = molecula.pos
n_energy_sample = molecula.y
# Hacer la inferencia
n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
# Transformar la inferencia
#n_energy_inferred = float(n_energy_inferred)
#n_energy_sample = float(n_energy_sample)

In [ ]:


# Cargar archivo splits
splits_data = np.load(path_splits_npy)

In [ ]:
# Acceder a la matriz específica dentro del archivo npz
splits = splits_data['idx_test'] 

# Indexar el dataset con los índices de splits
dataset_torchmdnet_test = torchmdnet_dataset[splits]

In [ ]:
if not os.path.exists('data/inferencias_test.dat'):
    with open('data/inferencias_test.dat', 'w') as file:
        for molecula in dataset_torchmdnet_test:
            z_sample = molecula.z
            pos_sample = molecula.pos
            n_energy_sample = molecula.y
            # Hacer la inferencia
            n_energy_inferred, n_forces_inferred = model(z_sample, pos_sample)
            # Transformar la inferencia
            n_energy_inferred = float(n_energy_inferred)
            n_energy_sample = float(n_energy_sample)        
            # Guardar los datos en el archivo
            file.write(f"{n_energy_sample} {n_energy_inferred}\n")
else:
    print("El archivo 'inferencias_test.dat' ya existe. No se ha realizado ninguna acción.")